# Python-C interoperability

Python has an [excellent C API](https://docs.python.org/3/c-api/index.html) for writing extension modules in C/C++. Let's not use it.

In [31]:
with open("tmp.c", "w") as file:
    file.write("""
#include <math.h>
void runs_in_c(int N, double* p, double* px, double* py, double* pz) {
    int i;
    for (i = 0;  i < N;  i++) {
        p[i] = sqrt(px[i]*px[i] + py[i]*py[i] + pz[i]*pz[i]);
    }
}""")
import os
assert os.system("gcc -fPIC -shared tmp.c -o libtmp.so") == 0

In [32]:
import ctypes
libtmp = ctypes.cdll.LoadLibrary("./libtmp.so")

In [33]:
libtmp.runs_in_c.argtypes = (ctypes.c_int, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p)
libtmp.runs_in_c.restype = None

In [34]:
import numpy
px = numpy.random.normal(0, 10, 1000000)
py = numpy.random.normal(0, 10, 1000000)
pz = numpy.random.normal(0, 10, 1000000)
p = numpy.zeros(1000000)

libtmp.runs_in_c(1000000, p.ctypes.data, px.ctypes.data, py.ctypes.data, pz.ctypes.data)

In [37]:
p

array([ 4.86754537, 29.86413297, 16.75806587, ...,  9.89957211,
       25.83591947, 23.8126184 ])

**Going the other way:** running Python in C.

The GNU Scientific Library is a large collection of C functions, some of which take functions as arguments:

```c
struct gsl_function {
    double (*function) (double x, void* params);
    void* params;
};
```

Let's convince GSL to take a Python function as though it were a C function (in its derivative-calculating method).

In [46]:
def f(x, params):
    out = 3*x**2 + 2*x + 1
    print("f({}) = {}".format(x, out))
    return out

In [47]:
# express the (double, double -> void) function type in ctypes
func_type = ctypes.CFUNCTYPE(ctypes.c_double, ctypes.c_double, ctypes.c_void_p)

# express the gsl_function struct in ctypes
class gsl_function(ctypes.Structure):
    _fields_ = [("f", func_type), ("params", ctypes.c_void_p)]

# make a ctypes object for the function
callback = gsl_function(func_type(f), 0)

In [52]:
gslcblas = ctypes.CDLL("libgslcblas.so", mode=ctypes.RTLD_GLOBAL)
gsl = ctypes.CDLL("libgsl.so")

# pointers to GSL's output: result and abserr
p_result = ctypes.POINTER(ctypes.c_double)(ctypes.c_double(-1))
p_abserr = ctypes.POINTER(ctypes.c_double)(ctypes.c_double(-1))

# call the function
assert gsl.gsl_deriv_central(
    ctypes.POINTER(gsl_function)(callback), ctypes.c_double(2), ctypes.c_double(1e-8), p_result, p_abserr) == 0

f(1.99999999) = 16.999999860000003
f(2.00000001) = 17.000000139999997
f(1.999999995) = 16.99999993
f(2.000000005) = 17.00000007


In [53]:
p_result[0], p_abserr[0]

(13.999999737279722, 2.886579852358283e-06)

**Doing it in ROOT:** ROOT has a built-in compiler and PyROOT automatically generates bindings.

In [1]:
import ROOT

Welcome to JupyROOT 6.10/02


In [5]:
ROOT.gInterpreter.Declare("""
void runs_in_cpp(int N, double* p, double* px, double* py, double* pz) {
    int i;
    for (i = 0;  i < N;  i++) {
        p[i] = sqrt(px[i]*px[i] + py[i]*py[i] + pz[i]*pz[i]);
    }
}""")

True

In [6]:
import numpy
px = numpy.random.normal(0, 10, 1000000)
py = numpy.random.normal(0, 10, 1000000)
pz = numpy.random.normal(0, 10, 1000000)
p = numpy.zeros(1000000)

ROOT.runs_in_cpp(1000000, p, px, py, pz)

In [7]:
p

array([ 6.86163698,  7.71003888, 26.79174061, ..., 18.24302702,
       16.27478819, 11.20169216])

**To mix Python objects with C++ objects, use Cython.**

# PyCUDA (and PyOpenCL): arbitrary GPU programming

The lowest of low-level programming— programming and running accelerator devices— can be done from the comfort of a notebook.

In [ ]:
import pycuda
import pycuda.autoinit
import pycuda.driver
import pycuda.compiler

compiled_cuda = pycuda.compiler.SourceModule("""
__global__ void runs_on_gpu(float* p, float* px, float* py, float* pz) {
    const int i = blockDim.x * blockIdx.x + threadIdx.x;
    if (i < 1000000) {
        p[i] = sqrt(px[i]*px[i] + py[i]*py[i] + pz[i]*pz[i]);
    }
}""")
runs_on_gpu = compiled_cuda.get_function("runs_on_gpu")

In [ ]:
import numpy

px = pycuda.driver.In(numpy.random.normal(0, 10, 1000000).astype(numpy.float32))
py = pycuda.driver.In(numpy.random.normal(0, 10, 1000000).astype(dtype=numpy.float32))
pz = pycuda.driver.In(numpy.random.normal(0, 100, 1000000).astype(dtype=numpy.float32))
p = numpy.zeros(1000000, dtype=numpy.float32)

runs_on_gpu(pycuda.driver.Out(p), px, py, pz, block=(1024, 1, 1), grid=(1000000 // 1024 + 1, 1))

In [ ]:
p

Working with Numpy arrays is already GPU-like.

In [ ]:
import pycuda.gpuarray

px = pycuda.gpuarray.to_gpu(numpy.random.normal(0, 10, 1000000))
py = pycuda.gpuarray.to_gpu(numpy.random.normal(0, 10, 1000000))
pz = pycuda.gpuarray.to_gpu(numpy.random.normal(0, 100, 1000000))

type(px)

In [ ]:
p = (px**2 + py**2 + pz**2)**0.5
type(p)

In [ ]:
p.get()

Isn't that what CuPy does? Yes, yes it is.

Once, there was also a "gnumpy" (U Toronto CS dept). Like Numeric and numarray, this is an active area of development.

# Numba-CUDA: compile *Python* code on the GPU

Numba can compile a subset of Python for CPUs with LLVM or GPUs with CUDA.

In [ ]:
import math
import numba.cuda

@numba.cuda.jit
def runs_on_gpu(p, px, py, pz):
    i = numba.cuda.grid(1)
    p[i] = math.sqrt(px[i]**2 + py[i]**2 + pz[i]**2)

In [ ]:
px = numpy.random.normal(0, 10, 1000000)
py = numpy.random.normal(0, 10, 1000000)
pz = numpy.random.normal(0, 100, 1000000)
p = numpy.zeros(1000000)

runs_on_gpu(p, px, py, pz)

In [ ]:
p

# 